# make barcode

# pair adapter
input : 
- forward raw fastq files
- reverse raw fastq files
- barcode fasta
output :
- demux R1
- demux R2

# reorientation
input :
- demux R1
- demux R2
- forward primer sequence

output :
- tmp
- reorient R1
- reorient R2

# cleaning
input :
- reorient R1
- reorient R2
- reverse primer sequence

output :
- clean R1
- clean R2


In [1]:
import pandas as pd
import os

import tempfile

from shutil import copyfile

In [10]:
! rm -r ../data/preprocessing/QC-Demux/*
for i in range(3):
    #! fastqc -q -t 8 ../data/preprocessing/Psoil-{i+1}/demux/*.fastq.gz 
    ! multiqc ../data/preprocessing/Psoil-{i+1}/demux/ -o ../data/preprocessing/QC-Demux

[WARNING]         multiqc : MultiQC Version v1.10.1 now available!
[INFO   ]         multiqc : This is MultiQC v1.10
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /home/WIN.DTU.DK/matinnu/phd/projects/dyrehavn16s/data/preprocessing/Psoil-1/demux
Searching   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 190/190  oil-1-demux-cutadapt-checkpoint.log.html
[INFO   ]        cutadapt : Found 1 reports
[INFO   ]          fastqc : Found 62 reports
[INFO   ]         multiqc : Compressing plot data
[INFO   ]         multiqc : Report      : ../data/preprocessing/QC-Demux/multiqc_report.html
[INFO   ]         multiqc : Data        : ../data/preprocessing/QC-Demux/multiqc_data
[INFO   ]         multiqc : MultiQC complete
[WARNING]         multiqc : MultiQC Version v1.10.1 now available!
[INFO   ]         multiqc : This is MultiQC v1.10
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /home/WIN.DTU.DK/matinnu/phd

In [13]:
#path = '../data/test/demux/'
#outdir = '../data/test/out'
#primer = 'CCTACGGGNGGCWGCAG'

def reorient(path, outdir, primer):
    try:
        os.mkdir(outdir)
    except FileExistsError:
        pass

    def make_manifest(path, out):
        manifest = sorted([i for i in os.listdir(path) if i.endswith('.fastq.gz')])
        ctr = 0
        df = pd.DataFrame(columns=['sample-id','forward-absolute-filepath','reverse-absolute-filepath'])

        for i in range(int(len(manifest)/2)):
            forward = manifest[ctr]
            reverse = manifest[ctr+1]
            sampleid = (manifest[ctr].rsplit('_', 4)[0])
            df.loc[i] = [sampleid, os.path.join(path, forward), os.path.join(path, reverse)]
            ctr = ctr+2
        df.to_csv(out, sep='\t', index=False)
        return

    manifest = os.path.join(outdir, 'manifest.txt')
    make_manifest(path, manifest)

    #
    try:
        os.mkdir(os.path.join(outdir, 'reorient'))
    except FileExistsError:
        pass

    df = pd.read_csv(manifest, sep='\t')
    for i in range(len(df)):
        # reorientation
        with tempfile.TemporaryDirectory() as tmp:
            name = df.loc[i, 'forward-absolute-filepath'].split('/')[-1].rsplit('_', 3)[0]
            print(name)
            shell = '''cutadapt \
                -j 0 \
                -e 0.1 \
                -g '''+name+'''='''+primer+''' \
                -o '''+tmp+'''/round1-{name}.R1.fastq.gz \
                -p '''+tmp+'''/round1-{name}.R2.fastq.gz \
                '''+df.loc[i, 'reverse-absolute-filepath']+''' \
                '''+df.loc[i, 'forward-absolute-filepath']+''' \
                > '''+tmp+'''/'''+name+'''-round1.log
                '''
            ! {shell}

            shell ='''cutadapt \
                -j 0 \
                -e 0.1 \
                -g '''+name+'''='''+primer+''' \
                -o '''+tmp+'''/round2-{name}.R1.fastq.gz \
                -p '''+tmp+'''/round2-{name}.R2.fastq.gz \
                '''+tmp+'''/round1-unknown.R2.fastq.gz \
                '''+tmp+'''/round1-unknown.R1.fastq.gz \
                > '''+tmp+'''/'''+name+'''-round2.log
                '''
            ! {shell}
            
            shell = '''cat '''+tmp+'''/round1-'''+name+'''.R1.fastq.gz '''+tmp+'''/round2-'''+name+'''.R1.fastq.gz > '''+outdir+'''/reorient/'''+name+'''.R1.fastq.gz'''
            #print(shell)
            ! {shell}
 
            shell = '''cat '''+tmp+'''/round1-'''+name+'''.R2.fastq.gz '''+tmp+'''/round2-'''+name+'''.R2.fastq.gz > '''+outdir+'''/reorient/'''+name+'''.R2.fastq.gz'''
            ! {shell}

            shell = '''cat '''+tmp+'''/'''+name+'''-round1.log '''+tmp+'''/'''+name+'''-round2.log > '''+outdir+'''/reorient/'''+name+'''.log'''
            ! {shell} 
    return

In [14]:
for i in range(3):
    path = '../data/preprocessing/Psoil-'+str(i+1)+'/demux/'
    outdir = '../data/preprocessing/Psoil-'+str(i+1)+'/'
    primer = 'CCTACGGGNGGCWGCAG'
    reorient(path, outdir, primer)

mkdir: missing operand
Try 'mkdir --help' for more information.
S1_rep1
[8<----------] 00:00:00        58,356 reads  @     12.2 µs/read;   4.94 M reads/minute
[8<----------] 00:00:00        41,708 reads  @     14.2 µs/read;   4.23 M reads/minute
cat /tmp/tmp9ly8_cyd/round1-S1_rep1.R1.fastq.gz /tmp/tmp9ly8_cyd/round2-S1_rep1.R1.fastq.gz > ../data/preprocessing/Psoil-1//reorient/S1_rep1.R1.fastq.gz
S1_rep2
[8=----------] 00:00:01       184,683 reads  @      8.2 µs/read;   7.36 M reads/minute
[8<----------] 00:00:00       126,201 reads  @      7.6 µs/read;   7.84 M reads/minute
cat /tmp/tmpgsqzsg9e/round1-S1_rep2.R1.fastq.gz /tmp/tmpgsqzsg9e/round2-S1_rep2.R1.fastq.gz > ../data/preprocessing/Psoil-1//reorient/S1_rep2.R1.fastq.gz
S1_rep3
[8<----------] 00:00:01       118,356 reads  @      9.2 µs/read;   6.53 M reads/minute
[8<----------] 00:00:00        80,969 reads  @     10.9 µs/read;   5.53 M reads/minute
cat /tmp/tmpjrshayuf/round1-S1_rep3.R1.fastq.gz /tmp/tmpjrshayuf/round2-S1_rep3.R1

In [15]:
! rm -r ../data/preprocessing/QC-Reorient/*
for i in range(3):
    ! fastqc -q -t 8 ../data/preprocessing/Psoil-{i+1}/reorient/*.fastq.gz 
    ! multiqc ../data/preprocessing/Psoil-{i+1}/reorient/ -o ../data/preprocessing/QC-Reorient

java.util.zip.ZipException: Not in GZIP format
	at java.base/java.util.zip.GZIPInputStream.readHeader(GZIPInputStream.java:166)
	at java.base/java.util.zip.GZIPInputStream.<init>(GZIPInputStream.java:80)
	at java.base/java.util.zip.GZIPInputStream.<init>(GZIPInputStream.java:92)
	at uk.ac.babraham.FastQC.Utilities.MultiMemberGZIPInputStream.<init>(MultiMemberGZIPInputStream.java:43)
	at uk.ac.babraham.FastQC.Utilities.MultiMemberGZIPInputStream.read(MultiMemberGZIPInputStream.java:103)
	at java.base/sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at java.base/sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at java.base/sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.base/java.io.InputStreamReader.read(InputStreamReader.java:185)
	at java.base/java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:326)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:392)
	at uk.ac.babr

# clean reverse primer & length
! cutadapt \
    -j 8 \
    -e 0.1 \
    -m 150 \
    -G GACTACHVGGGTATCTAATCC \
    -o ../data/sub_sampling/clean-P5_rep1-Psoil-2_S2_L001_R1_001.fastq.gz \
    -p ../data/sub_sampling/clean-P5_rep1-Psoil-2_S2_L001_R2_001.fastq.gz \
    ../data/sub_sampling/reoriented-P5_rep1-Psoil-2_S2_L001_R1_001.fastq.gz \
    ../data/sub_sampling/reoriented-P5_rep1-Psoil-2_S2_L001_R2_001.fastq.gz

In [16]:
#path = '../data/test/demux/'
#outdir = '../data/test/out'
#primer = 'CCTACGGGNGGCWGCAG'

def clean(path, outdir, primer):
    try:
        os.mkdir(outdir)
    except FileExistsError:
        pass

    def make_manifest(path, out):
        manifest = sorted([i for i in os.listdir(path) if i.endswith('.fastq.gz')])
        ctr = 0
        df = pd.DataFrame(columns=['sample-id','forward-absolute-filepath','reverse-absolute-filepath'])

        for i in range(int(len(manifest)/2)):
            forward = manifest[ctr]
            reverse = manifest[ctr+1]
            sampleid = (manifest[ctr].rsplit('_', 4)[0])
            df.loc[i] = [sampleid, os.path.join(path, forward), os.path.join(path, reverse)]
            ctr = ctr+2
        df.to_csv(out, sep='\t', index=False)
        return

    manifest = os.path.join(outdir, 'manifest.txt')
    make_manifest(path, manifest)

    #
    try:
        os.mkdir(os.path.join(outdir, 'clean'))
    except FileExistsError:
        pass

    df = pd.read_csv(manifest, sep='\t')

    for i in range(len(df)):
        # reorientation
        name = df.loc[i, 'forward-absolute-filepath'].split('/')[-1].rsplit('.', 3)[0]
        print(name)
        #print(df.loc[i, 'forward-absolute-filepath'])
        shell = '''cutadapt \
                -j 0 \
                -e 0.1 \
                -G '''+primer+''' \
                -o '''+outdir+'''/clean/'''+name+'''.R1.fastq.gz \
                -p '''+outdir+'''/clean/'''+name+'''.R2.fastq.gz \
                '''+df.loc[i, 'forward-absolute-filepath']+''' \
                '''+df.loc[i, 'reverse-absolute-filepath']+''' \
                > '''+outdir+'''/clean/'''+name+'''.log
                '''
        ! {shell}
        #print(shell)
    return 
for i in range(3):
    path = '../data/preprocessing/Psoil-'+str(i+1)+'/reorient/'
    outdir = '../data/preprocessing/Psoil-'+str(i+1)
    primer = 'GACTACHVGGGTATCTAATCC'
    clean(path, outdir, primer)

S1_rep1
[8<----------] 00:00:00        35,104 reads  @     17.0 µs/read;   3.54 M reads/minute
S1_rep2
[8=----------] 00:00:01       115,009 reads  @     13.2 µs/read;   4.54 M reads/minute
S1_rep3
[8<----------] 00:00:00        73,253 reads  @     12.6 µs/read;   4.77 M reads/minute
S1_rep4
[8=----------] 00:00:01       103,918 reads  @     12.2 µs/read;   4.90 M reads/minute
S1_rep5
[8=----------] 00:00:01       133,619 reads  @     12.5 µs/read;   4.78 M reads/minute
S2_rep1
[8<----------] 00:00:00        47,882 reads  @     14.8 µs/read;   4.05 M reads/minute
S2_rep2
[8<----------] 00:00:00        38,005 reads  @     15.1 µs/read;   3.98 M reads/minute
S2_rep3
[8=----------] 00:00:01       171,488 reads  @     11.0 µs/read;   5.47 M reads/minute
S2_rep4
[8<----------] 00:00:00        54,504 reads  @     14.5 µs/read;   4.13 M reads/minute
S2_rep5
[8<----------] 00:00:00        38,471 reads  @     16.0 µs/read;   3.75 M reads/minute
S3_rep1
[8<----------] 00:00:00        45,752 read

In [17]:
! rm -r ../data/preprocessing/QC-Clean/*
for i in range(3):
    ! fastqc -q -t 8 ../data/preprocessing/Psoil-{i+1}/clean/*.fastq.gz 
    ! multiqc ../data/preprocessing/Psoil-{i+1}/clean/ -o ../data/preprocessing/QC-Clean

[WARNING]         multiqc : MultiQC Version v1.10.1 now available!
[INFO   ]         multiqc : This is MultiQC v1.10
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /home/WIN.DTU.DK/matinnu/phd/projects/dyrehavn16s/data/preprocessing/Psoil-1/clean
Searching   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 222/222  soil-1/clean/S3_rep3.R2_fastqc.html.html
[INFO   ]        cutadapt : Found 31 reports
[INFO   ]          fastqc : Found 62 reports
[INFO   ]         multiqc : Compressing plot data
[INFO   ]         multiqc : Report      : ../data/preprocessing/QC-Clean/multiqc_report.html
[INFO   ]         multiqc : Data        : ../data/preprocessing/QC-Clean/multiqc_data
[INFO   ]         multiqc : MultiQC complete
[WARNING]         multiqc : MultiQC Version v1.10.1 now available!
[INFO   ]         multiqc : This is MultiQC v1.10
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /home/WIN.DTU.DK/matinnu/ph

In [6]:
# remove temp file to save space
for i in range(3):
    ! find ../data/preprocessing/Psoil-{i+1}/reorient/ -name "*.fastq.gz" -type f -delete
    ! find ../data/preprocessing/Psoil-{i+1}/demux/ -name "*.fastq.gz" -type f -delete